# Word Piece Model

토크나이징(tokenizing)은 문장을 토큰으로 나누는 과정입니다.  
텍스트 데이터를 학습한 모델의 크기는 단어의 개수에 영향을 받습니다.  
특히 Google neural machine translator 와 같은 RNN 기반 알고리즘들은 단어 개수에 비례하여 계산 비용이 증가합니다.  
그렇기 떄문에 RNN 에 이용되는 vocabulary, word embedding 벡터의 종류가 제한됩니다. 하지만 vocabulary의 개수가 제한되면 임베딩 벡터로 표현하지 못하는 단어가 생깁니다. 이역시 미등록 단어 문제입니다. RNN 기반 모델에서 이를 해결하기 위하여 Word piece Model이 제안되었습니다. 단어를 한정적인 유닛(finite subword units)으로 표현합니다. WPM은 언어에 상관없이 모두 적용할 수 있기 때문에 적용할 언어마다 해당 언어의 특징을 반영한 토크나이저를 만들지 않아도 됩니다. 그러나 모든 데이터 분석에 적합한 것은 아닙니다. WPM이 유용할 수 있는 상황과 그렇지 않은 상황에 대해서 알아봅시다.  


### Word piece, unit of words
Word Piece Model은 제한적인 vocabulary units 정확히는 단어를 표현할 수 잇는 sub words units으로 모든 단어를 표현합니다, 수만개의 단어를 포함하는 데이터를 표현하기 위해서 bag of words model는 단어 개수 만큼의 차원을 지닌 벡터 공간을 이용합니다 .   
RNN 처럼 word embedding vetors 를 이용하는 모델은 단어 개수 만큼의 embedding vector를 학습합니다. 단어의 개수가 많을수록 차원이 크거나 모델이 무거워집니다. 이를 해결하기 위해서는 제한된 (finite)개수의 단어를 이용해야 합니다. 그러나 자주 이용되지 않는 수많은 (log-tail)단어들을 무시하면 미등록단어 문제가 발생합니다.  

언어는 글자(character)를 subword units으로 이용합니다. 영어는 알파벳을 유닛으로 이용합니다. 대부분의 영어 단어는 몇 개의 글자가 모여 하나의 단어를 구성합니다. 즉, 하나의 유닛이 어떤 개념을 지칭하기는 어렵습니다. 유닛이 모호성을 지닙니다. 중국어는 한자를 units로 이용합니다. 중국어도 여러 글자가 모여 하나의 단어를 이루기도 하지만 , 한 글자로 구성된 단어도 많습니다. 영어보다는 유닛의 모호서이 줄어듭니다. 동음이의어의 문제가 남아있지만, 가장 모호성이 적은 방법은 모든 단어를 단어의 유닛으로 이용하는 것 입니다.  

그러나 토크나이징 방법에 따라 모호성이 적은 최소한의 유닛을 만들수도 있습니다.  
아래의 세문장을 다음의 요소들로 나눈다면 이 유닛들은 의미를 보존하면서도 재활용이 돌 수 있습니다.

공연은 끝났어 -> ['공연-' + '-은' + '끝-' + '-났어']
공연을 끝냈어 -> ['공연-' + '-을' + '끝-' + '-냈어']
개막을 해냈어 -> ['개막-' + '-을' + '해-' + '-냈어']


이 유닛들은 '개막공연'이라는 복합명사도 분해하는데 이용될 수 있습니다. '개막공연'을 독립된 유닛으로 만들 필요가 없습니다.  

개막공연을 끝냈어 -> ['개막-' + '공연-' + '-을' + '끝-' + '-냈어']  

그런데 문제는 위처럼 토크나이징을 하려면 해당 언어의 언어학적 지식과 학습데이터가 필요합니다. 그러나 언어가 다르고, 도메인이 다르면 이를 준비하는 것은 어렵습니다.

### Word Piece Model (sentencepiece) tokenizer

학습데이터를 이용하지 않으면서도 위의 결과를 이끌 수 있는 heruistic이 있습니다. '공연', '개막', '냈어', '났어'는 유닛이기 때문에 유닛이 아닌 subwords 보다 자주 등장할 가능성이 높습니다. 만약 '공연'의 빈도수가 '개막공연'의 빈도수가 같다면 '공연'이나 '개막'은 유닛으로 이용하지 않아도 됩니다. 어차피 세 유닛 모두 '개막공연'을 나타내기 위한 부분들일테니깐요, 유닛이 자주 등장한다는 사실은 아마도 많은 언어의 공통적인 특징일 것입니다..  
이를 이용한다면 laguage independent, universial tokenizer 를 만들 수도 있을 것 같습니다.  

Word Piece Model(WPM)은 이 개념을 이용하는 토크나이저입니다.  
원 논문에 적힌 예시입니다.  
Words 는 매우 다양합니다. makers, over는 모두 자주 이용되기 때문에 그 자체를 units로 이용합니다, jet은 자주 등장하지 않는 단어이기 때문에 subword units인 j와 et로 나눕니다,  

그 전에 모든 단어의 시작에는 underbar '_'를 붙입니다.  그렇다면 jet는 _jet가 되어 나누어집니다.
makers는 "_makers"가 됩니다.  

Word : Jet makers feud over seat width with big orders at stake  
Wordpieces: _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake  

 Underbar는 문장 생성, 혹은 subwords 부터의 문장 복원을 위한 특수기호 입니다. Underbar 없이 subwords를 띄어두면, 본래 띄어쓰기와 구분이 되지 않기 때문입니다 문장 복원을하는 코드는 간단합니다. 띄어쓰기 기준으로 나눠진 tokens를 concatenation 한뒤, _로 다시 나눠 tokenize하거나 _를 빈 칸으로 치환하여 문장으로 복원합니다.  


In [1]:
def recover(tokens):
    sent = ''.join(tokens)
    sent = sent.replace('_', ' ')
    return sent

### Byte-pair Encoding (BPE)

Neural Machine Translation of Rare Words with Subword Units (Sennrich et al., 2015) 에는 word pieces (subword units) 을 학습할 수 있는 간단한 코드가 적혀 있습니다.  

아래의 vocab 은 low, lower, newest, widest 의 맨 뒤에 특수기호 ‘/w’ 를 넣은 뒤, 한글자 단위로 모두 띄어 초기화를 한 상태입니다. Character 는 기본 subword units 입니다. for loop 에서 빈도수가 가장 많은 bigram 을 찾습니다. 이 bigram 을 하나의 unit 으로 merge 합니다. 이 과정을 num_merges 만큼 반복합니다. vocab 의 value 는 빈도수 입니다. ‘low’ 가 5 번, ‘lower’ 가 2 번 등장했습니다.

In [2]:
import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best)

print(vocab)

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


위 코드를 실행시킬 때 print(best)에 의하여 출력된 결과입니다. 처음 'es'는 9번 등장 하였기 떄문에 ('e','s')가 'es'로 합쳐집니다. 'lo'는 7번 등장했습니다만 Python dict의 key order순서에 의해 'es'가 우선적으로 merge됩니다. 그리고 ('es','t')가 병합되어 'est'가 됩니다. 같은 빈도수를 지닌다면 계속하여 병합이 됩니다. 하지만 .west가 6번 dest가 3번 이기 때문에 est이후에선 ('l','o'),('lo','w')가 병합됩니다.

vocab = {
         'low</w>': 5, 
         'low e r </w>': 2, 
         'newest</w>': 6, 
         'wi d est</w>': 3
        }

띄어쓰기 기준으로 subword unit를 나누면 다음과 같습니다. 총 9개의 unit으로 네 단어를 표현합니다 

{'low</w>': 5,
 'low': 2,
 'e': 2,
 'r': 2,
 '</w>': 2,
 'newest</w>': 6,
 'wi': 3,
 'd': 3,
 'est</w>': 3}

Byte pair encoding 은 데이터 압축 방법입니다. 빈도수가 많은 최장길이의 substring 을 하나의 unit 으로 만들면 bit 를 아낄 수 있습니다.  

토크나이저 입장에서는 많이 쓰이는 subwords 를 units 으로 이용하면 자주 이용되는 단어는 그 자체가 unit이 되며 자주 등장하지 않는 다어(rare words)가 subword unit으로 나뉘어 집니다. 즉 WPM은 각 언어의 지식이 없이도 빈번히 등장하는 substring을 단어로 학습하고, 자주 등장하지 않는 단어들을 최대한 의미보존을 할 수 있는 최소한의 units으로 표현합니다. 한국에서 복합명사를 단일 명사들로, 어절을 명사와 조사로 나누는 것과 비슷합니다,  

다른 논문들을 살펴보면, 번역 엔진들에서는 자주 이용되는 k1개의 단어는 따로 지정을 하고 그 외의 단어에 대해서 k2개의 subword units을 이용하여 토크나이징을 수행한다고 합니다. 


논문의 원저자인 Sennrich 도 본인의 github 에 코드를 공개하였습니다. Python script 형식으로 input file 을 토크나이징하여 output file 로 만듭니다.  

Codes (논문)
Sennrich et al (2015) 에 공개된 코드에 save, load 와 같은 기능을 추가하여 간단히 word piece model 을 구현하였습니다. 코드는 github에 공개하였습니다.  

BytePairEncoder(n_units) 은 n_units 개수 만큼의 subword units 을 학습하는 클래스입니다. corpus 는 list of str (like) 입니다.

In [2]:
from bytepairencoder import BytePairEncoder

n_units = 5000
encoder = BytePairEncoder(n_units)
encoder.train(corpus)

ModuleNotFoundError: No module named 'bytepairencoder'

In [3]:
tokens = encoder.tokenize(sent)

NameError: name 'encoder' is not defined

In [4]:
encoder.save(model_path)

loaded_encoder = BytePairEncoder()
loaded_encoder.load(model_path)

NameError: name 'encoder' is not defined

In [5]:
sent = '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다'

encoder.tokenize(sent)

NameError: name 'encoder' is not defined

오패산터널의 경우, 잘 등장하지 않은 단어이기 때문에 모든 단어가 글자들로 나뉘어 집니다. ‘용의자’는 빈도수 상위 5000 등 안에 들지 못하였습니다. 그러나 ‘의자’는 5000 등 안에 들었기 때문에 하나의 subword unit 이 되었습니다. ‘연합뉴스’는 뉴스 문서에서는 매우 빈번한 단어이기 때문에 하나의 단어임을 확인할 수 있습니다.  

의미적으로 ‘용’ + ‘의자’ = ‘용의자’는 아닙니다. 하지만 composition 을 통하여 ‘용의자’의 의미를 파악하는 것은 더 이상 토크나이저의 몫은 아닙니다. 이 부분은 번역기, 문서 판별기 등의 알고리즘의 몫입니다.